<a href="https://colab.research.google.com/github/CarolinB/Active-Gaming-TechLabs-2022/blob/main/GoogLeNet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
#Install PyTorch
!pip3 install torch torchvision

In [32]:
#Import modules

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision

In [33]:
#Dataset builder modules
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io
from skimage import color
from PIL import Image

In [34]:
#writing builder
class data_builder(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        #image = io.imread(img_path, as_grey=True)
        image = Image.open(img_path)
        image = image.convert("RGB")
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [35]:
#set_device

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [36]:
# Hyperparameters
in_channel = 3
num_classes = 4
learning_rate = 1e-3
batch_size = 32
num_epochs = 10


In [37]:
#Connect colab to Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [38]:
# Load data

transform = transforms.Compose(
    [transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    transforms.Resize((256, 256))
    ])


dataset = data_builder(csv_file = 'C:/Users/franc/Desktop/Techlabs-2022.1/Github/Active-Gaming-TechLabs-2022/parameters_full.csv', 
                       root_dir = 'C:/Users/franc/Desktop/Techlabs-2022.1/Github/Active-Gaming-TechLabs-2022/Arm positions images',
                        
                       transform = transform)

train_set, test_set = torch.utils.data.random_split(dataset, [230, 25  ])


train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [39]:
# Model
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

C:\Users\franc\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\franc\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [40]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [42]:
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

Cost at epoch 0 is 0.08762912184465677
Cost at epoch 1 is 0.02053723661811091
Cost at epoch 2 is 0.009614321344997734
Cost at epoch 3 is 0.004249542849720456
Cost at epoch 4 is 0.003620841176598333
Cost at epoch 5 is 0.0008915294220059877
Cost at epoch 6 is 0.0025857143173197983
Cost at epoch 7 is 0.1297010343441798
Cost at epoch 8 is 0.11685702000249876
Cost at epoch 9 is 0.06832066364586353


In [43]:
#checking the channels of the images
for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
    data = data.to(device=device)
    print(data.shape)

torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])
torch.Size([32, 3, 256, 256])


KeyboardInterrupt: 

In [48]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Checking accuracy on Training Set
Got 226 / 230 with accuracy 98.26
Checking accuracy on Test Set
Got 23 / 25 with accuracy 92.00


In [49]:
#saving model with state_dict

#Path = '/content/drive/MyDrive/AI group 2_Active Gaming/model.pth'

#print("Our model: \n\n", model, '\n')
#print("The state dict keys: \n\n", model.state_dict().keys())

#torch.save(model.state_dict(), Path)

In [50]:
#load model with state_dict

#model = torchvision.models.googlenet(pretrained=True)
#model.load_state_dict(torch.load(Path))
#model.eval()

#print("Our model: \n\n", model, '\n')
#print("The state dict keys: \n\n", model.state_dict().keys())

In [51]:
#saving complete model

#Path = '/content/drive/MyDrive/AI group 2_Active Gaming/complete_model.pt'

#torch.save(model, Path)